# Insulin cystallization study

Prepared 10/12/2018

In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import skimage
from skimage import io, exposure, img_as_float, img_as_ubyte, morphology, filters, util
from skimage.color import rgb2gray, label2rgb
from skimage.filters import threshold_minimum, sobel
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, watershed, square

import pandas as pd

import plotly as py
import plotly.graph_objs as go
import plotly.io as pio
from plotly import tools


from joblib import Parallel, delayed

import os

import itertools

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# User defined variables
## Name of the directory containing the images
dirname = "sample-images"

## Scale
scale = 659 #px/mm

In [3]:
# Create a list of the images to iterate on
image_list = []

for file in os.listdir(dirname):
    if file.endswith(".JPG"):
        image_list.append(file)
        
image_list.sort();

print(image_list)
print(len(image_list))

['DSC_6751.JPG', 'DSC_6752.JPG', 'DSC_6753.JPG', 'DSC_6754.JPG', 'DSC_6755.JPG', 'DSC_6756.JPG', 'DSC_6757.JPG', 'DSC_6758.JPG', 'DSC_6759.JPG', 'DSC_6760.JPG', 'DSC_6761.JPG', 'DSC_6762.JPG', 'DSC_6763.JPG', 'DSC_6764.JPG', 'DSC_6765.JPG', 'DSC_6766.JPG', 'DSC_6767.JPG', 'DSC_6768.JPG', 'DSC_6769.JPG', 'DSC_6770.JPG', 'DSC_6771.JPG', 'DSC_6772.JPG', 'DSC_6773.JPG', 'DSC_6774.JPG', 'DSC_6775.JPG', 'DSC_6776.JPG', 'DSC_6777.JPG', 'DSC_6778.JPG', 'DSC_6779.JPG', 'DSC_6780.JPG', 'DSC_6781.JPG', 'DSC_6782.JPG', 'DSC_6783.JPG', 'DSC_6784.JPG', 'DSC_6785.JPG', 'DSC_6786.JPG', 'DSC_6787.JPG', 'DSC_6788.JPG', 'DSC_6789.JPG', 'DSC_6790.JPG', 'DSC_6791.JPG', 'DSC_6792.JPG', 'DSC_6793.JPG', 'DSC_6794.JPG', 'DSC_6795.JPG', 'DSC_6796.JPG', 'DSC_6797.JPG', 'DSC_6798.JPG', 'DSC_6799.JPG', 'DSC_6800.JPG', 'DSC_6801.JPG', 'DSC_6802.JPG', 'DSC_6803.JPG', 'DSC_6804.JPG', 'DSC_6805.JPG', 'DSC_6806.JPG', 'DSC_6807.JPG', 'DSC_6808.JPG', 'DSC_6809.JPG', 'DSC_6810.JPG', 'DSC_6811.JPG', 'DSC_6812.JPG', 'DSC_68

In [4]:
#IO operations
def process(file,f_watershed):
    #Open the image
    image = io.imread(os.getcwd()+"/"+dirname+"/"+file)
    #Analyze the image
    if f_watershed:
        crystalProps = analyze_crystals_watershed(image)
    else:
        crystalProps = analyze_crystals(image)
    #Retrieve the functionalization name and picture number
    idx = image_list.index(file)
    funct = funct_order[idx//8]
    pic_num = idx%8
    # DEBUG # Save the processed image
    save_image(image, funct + '_' + str(pic_num), crystalProps, f_watershed)
    #Create a dataframe to add the crystals to
    col_names = ['Functionalization', 'Pic#', 'Cystal#', 'Size', 'Mean Intensity']
    df = pd.DataFrame(columns=col_names)
    #Drop the crystal information in the dataframe
    for i in range(len(crystalProps)):
        df.loc[i] = [funct, pic_num, i, crystalProps[i].area, crystalProps[i].mean_intensity]
    return df

#Image analysis function (Watershed)
def analyze_crystals_watershed(image):
    #Convert to grayscale
    i_grey = rgb2gray(image)
    #Crop the sides
    i_cropped = i_grey[0:2848,500:4000]
    #Convert to uint8
    i_uint = img_as_ubyte(i_cropped)
    #Contrast stretching
    p2, p98 = np.percentile(i_uint, (2, 98))
    i_contStretch = exposure.rescale_intensity(i_uint, in_range=(p2, p98))
    #Construct elevation map
    elevation_map = sobel(i_contStretch)
    #Create markers using extremes of the intensity histogram
    markers = np.zeros_like(i_uint)
    markers[i_uint < 70] = 1
    markers[i_uint > 120] = 2
    #Apply the watershed algorithm for segmentation
    i_segmented = watershed(elevation_map, markers)
    #Close open domains
    i_closed = closing(i_segmented, square(3))
    #Remove artifacts connected to image border
    i_cleared = clear_border(i_closed)
    #Label image regions
    label_image = label(i_cleared)
    image_label_overlay = label2rgb(label_image, image=i_cropped)
    #Extract region properties
    crystalProps = regionprops(label_image, intensity_image=i_cropped)
    
    #Return
    return crystalProps

#Save the processed image (for debugging purposes)
def save_image(image, name, regionProps, f_watershed):
    
    if f_watershed:
        algo = 'Watershed'
        direct = os.getcwd()+"/"+dirname+"/"+'ProcessedImagesWtshd'
        if not os.path.exists(direct): os.mkdir(direct)
    else:
        algo = 'Minimum Threshold'
        direct = os.getcwd()+"/"+dirname+"/"+'ProcessedImages'
        if not os.path.exists(direct): os.mkdir(direct)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(image[0:2848,500:4000])

    for region in regionProps:
        # take regions with large enough areas
        if region.area >= 10:
            # draw rectangle around segmented coins
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=1)
            ax.add_patch(rect)

    ax.set_axis_off()
    ax.set_title(algo+ ' - ' + name)
    plt.tight_layout()
    plt.savefig(direct+'/'+name+".jpg")

In [6]:
#PROCESSING WATERSHED
#If the images have been processed already, load from disk, otherwise, process
savepath = os.getcwd()+"/"+dirname+"/"+"ProcessedData_Watershed"+".pkl"

if os.path.isfile(savepath):
    df = pd.read_pickle(savepath)
    print("Data loaded from disk")
else:
    print("Did not find data, processing images")
    #Run the crystal analysis on all images in parallel
    df_list = Parallel(n_jobs=-2, verbose=10)(delayed(process)(file,True) for file in
                                              image_list)
    #Merge all the dataframes in one comprehensive crystal dataframe and reindex
    df = pd.concat(df_list).reset_index(drop=True)
    #Save dataframe to disk
    df.to_pickle(savepath)

Data loaded from disk


In [7]:
df.tail(5)

,Functionalization,Pic#,Cystal#,Size,Mean Intensity
44916,GQD 100,7,1697,360,0.419541
44917,GQD 100,7,1698,604,0.769376
44918,GQD 100,7,1699,84,0.445882
44919,GQD 100,7,1700,326,0.461125
44920,GQD 100,7,1701,329,0.428052


We define a list of labels to be used for plotting purposes

In [8]:
df.groupby(['Functionalization', 'Pic#']).size().index.levels

labels = itertools.product(df.groupby(['Functionalization', 'Pic#']).size().index.levels[0], df.groupby(['Functionalization', 'Pic#']).size().index.levels[1])
labels_list = []

for name in labels:
    labels_list.append(name[0] + ' - ' + str(name[1]))

## Data cleanup

### Remove small crystals

First, we remove the smallest of the crystals. The minimum threshold algorithm in particular is prone to detecting tiny domains in larger crystals as particles. After a visual determination, 50px^2 was determined to be a reasonnable cutoff size for both algoritms

Define a frame for the number of crystals:

In [9]:
#Create a template for the dataframe containing the number of crystals per drop
df_num = df.groupby(['Functionalization', 'Pic#']).size().to_frame()
df_num.columns = ['# Crystals']
df_num.head(10)

# Crystals
Functionalization Pic#            
Control           0            416
                  1           1170
                  2            530
                  3            761
                  4            541
                  5           1008
                  6            490
                  7            810
GQD 100           0            849
                  1           2021

In [11]:
## Create a dataframe containing the number of crystals per drop, including the drops with 0 crystals!
for funct in funct_order:
    for i in range(8):
        try:
            df_num.loc[funct,i] = df.groupby(['Functionalization', 'Pic#']).size().loc[funct,i]
        except:
            df_num.loc[funct,i] = 0
        
df_num.head(10)

# Crystals
Functionalization Pic#            
Control           0            468
                  1           1087
                  2            484
                  3            678
                  4            500
                  5            778
                  6            464
                  7            659
GQD 100           0           1103
                  1           1793

### Evaluate the number of crystals per drop

This allows us to drop outliers, where the exposure of the pictures did not permit a good measurement

In [12]:
data=[]

data.append(go.Bar(
    text = labels_list,
    y = df_num['# Crystals']))

go.FigureWidget(
    data,
    layout = go.Layout(
        width = 800,
        height = 500,
        xaxis=dict(title='Individual Drop', linecolor = 'black',linewidth = 2, mirror = True),
        yaxis=dict(title='# of crystals',linecolor = 'black',linewidth = 2, mirror = True),
        showlegend=False
    )
)

FigureWidget({
    'data': [{'text': [Control - 0, Control - 1, Control - 2, Control - 3, Control
            …

## Data Analysis

### Number of crystals per drop

**Hypothesis**: Addition of particles will increase the number of crystals per drop while reducting their mean size.

Let's plot the number of crystals per drop as a function of functionalization

In [13]:
data=[]

for i in range(len(funct_order)):
    data.append(go.Box(
        y=df.groupby(['Functionalization', 'Pic#']).size().loc[funct_order[i]],
        name=funct_order[i],
        boxpoints='all',
        boxmean=True
    ))
    
go.FigureWidget(
    data,
    layout = go.Layout(
        width = 600,
        height = 500,
        xaxis=dict(title='Functionalization', linecolor = 'black',linewidth = 2, mirror = True),
        yaxis=dict(title='# of crystals',linecolor = 'black',linewidth = 2, mirror = True),
        showlegend=False
    )
)

FigureWidget({
    'data': [{'boxmean': True,
              'boxpoints': 'all',
              'name': 'NHS 80'…

There are not enough crystals to be described meaningfully by a boxplot, let's create a bar graph of the average number of crystals per drop as a function of functionalization

In [17]:
data = []

data.append(go.Bar(
    y = [df.loc[funct_select[i]].mean().iloc[0] for i in range(len(funct_select))] ,
    x = funct_select,
    error_y=dict(
        type='data',
        array=[df.loc[funct_select[i]].std().iloc[0]/2 for i in range(len(funct_select))],
        visible=True
    )
))
    
fig = go.FigureWidget(
    data,
    layout = go.Layout(
        width = 600,
        height = 500,
        xaxis=dict(title='Functionalization', linecolor = 'black',linewidth = 2, mirror = True),
        yaxis=dict(title='# of crystals per drop', range = (0,900), linecolor = 'black',linewidth = 2, mirror = True),
        showlegend=False
    )
)
pio.write_image(fig, dirname + '-AvgNumCrystPerDrop.png')
fig

FigureWidget({
    'data': [{'error_y': {'array': [81.3100053147564, 69.90401582578377,
                      …

### Crystal size distribution

Let's see whether the introduction of the particles has an effect on the size of the crystals

In [15]:
df_gSize = df_clean.groupby('Functionalization')['Size'].apply(list)
df_gSize

Functionalization
Control    [631, 2083, 385, 72, 2231, 3212, 140, 624, 36,...
GQD 100    [2907, 329, 505, 2216, 505, 938, 354, 785, 679...
MAL 100    [1634, 1508, 1649, 5771, 4842, 1155, 1019, 177...
MAL 120    [219, 820, 42, 1094, 265, 778, 92, 384, 520, 2...
MAL 80     [367, 2106, 2945, 355, 268, 987, 489, 269, 303...
NHS 100    [74, 167, 81, 98, 5834, 232, 98, 105, 250, 94,...
NHS 120    [62, 30, 246, 1050, 285, 61, 184, 99, 39, 114,...
NHS 80     [38, 110, 244, 1953, 640, 1923, 538, 67, 53, 8...
Name: Size, dtype: object

In [18]:
data=[]

for i in range(len(funct_order)):
    data.append(go.Box(
        y = np.multiply(df_gSize.loc[funct_order[i]],(1/scale)**2),
        name=funct_order[i],
        boxmean=True
    ))
    
fig = go.FigureWidget(
    data,
    layout = go.Layout(
        width = 600,
        height = 500,
        xaxis=dict(title='Functionalization', linecolor = 'black',linewidth = 2, mirror = True),
        yaxis=dict(title='Size of crystals (mm2)',type='log', linecolor = 'black',linewidth = 2, mirror = True),
        showlegend=False
    )
)
pio.write_image(fig, dirname + '-SizeCrystPerDrop.png')
fig

FigureWidget({
    'data': [{'boxmean': True,
              'name': 'NHS 80',
              'type': 'box',
   …